In [1]:

import pickle
from openai import OpenAI


In [2]:
data_path = 'data/data.txt'
model_name = 'gpt3.5-two-shot-pool-V2-temp0'
path_output_pkl = 'outputs/'+model_name+'.pkl'
output_text_path = 'outputs/'+model_name+'.txt'
sample_passage_path = 'data/random_positive_negative_examples.txt'
model_id = "gpt-3.5-turbo-0125"
API_key = ""
FROM_CHECKPOINT= True

In [3]:
# Create the prompt per each turn
def create_data_two_shot():

    prompt = """You are a search quality rater evaluating the relevance of web pages. 
    Given the persona of the user, user query, and a web page, you must provide a score on an integer scale of 0 to 4 to indicate to what extent the given document meets the information needs of the user. The scores have the following meanings:

    0: fails to meet 
    1: slightly meets
    2: moderately meets
    3: highly meets
    4: fully meets

    User persona: {ptkb}
    Query: {utterance}

    Passage 1: {pass_1}
    Score: {score_1}

    Passage 2: {pass_2}
    Score: {score_2}

    Passage 3: {passage}
    Score:
    Please only generate an int score between 0 to 4 to say to what extent the document 3 is relevant to the user question. Score lower than 2 means the document is not relevant.
    """


    with open(sample_passage_path, 'r') as f:
        sample_passages_data = f.readlines()
    
    sample_passage_turn = {}
    
    for line_data in sample_passages_data:
        turn_id, pass_1, score_1, pass_2, score_2, _ = line_data.split('\t')
        y = {'pass_1': pass_1,
             'pass_2': pass_2,
             'score_1': score_1,
             'score_2': score_2}
        sample_passage_turn[turn_id] = y

    with open(data_path, 'r') as f:
        lines = f.readlines()

    data_points = []

    for line in lines:
        turn_id, rewritten_utterance, response_txt,  passage_id, passage_text, score,  ptkb, _ =line.split('\t')
        x = {'turn_id': turn_id,
        'doc_id': passage_id,
        'passage': passage_text,
        'score': score}

        x['prompt'] = prompt.format(ptkb=ptkb, utterance=rewritten_utterance, passage=passage_text, 
                                    pass_1 = sample_passage_turn[turn_id]['pass_1'],
                                    pass_2 = sample_passage_turn[turn_id]['pass_2'],
                                    score_1 = sample_passage_turn[turn_id]['score_1'],
                                    score_2 = sample_passage_turn[turn_id]['score_2'])
        data_points.append(x)
    
    return data_points

data_points = create_data_two_shot()

if FROM_CHECKPOINT:
    print('Loading from checkpoint')
    with open(path_output_pkl, 'rb') as f:
        data_points = pickle.load(f)



Loading from checkpoint


In [4]:
def chatgpt_conversation(conversation_Log):
  response = client.chat.completions.create(
      model = model_id,
      messages= conversation_Log,
      temperature= 0,
      top_p= 1
  )
  
  response= response.choices[0].message.content

  return response

def run_one_sample(prompt):

    conversations = []
    conversations.append({'role': 'user', 'content': prompt})
    response = chatgpt_conversation(conversations)
    score = response.strip()
    
    return score

client = OpenAI(api_key=API_key)


In [ ]:
counter = 0
index = 0

for entry in data_points:
    index += 1
    print(index)

    if ('gpt3.5-score' in entry) and (not(entry['gpt3.5-score']=='NONE')):
        print(entry['gpt3.5-score'])
        print('&&&&&&&&&&&&&&')
        continue
    
    else:
        try:
            pred_score = run_one_sample(entry['prompt'])
            pred_score = pred_score.lower().strip()
            entry['gpt3.5-score'] = pred_score
            print(entry['gpt3.5-score'])
            print('****************')
            counter+=1
        except:
            print('unsuccessful')
            entry['gpt3.5-score'] = 'NONE'

    if index%100==99:
        print('100 done')
        with open(path_output_pkl, 'wb') as f:
            pickle.dump(data_points, f)


with open(path_output_pkl, 'wb') as f:
    pickle.dump(data_points, f)            

In [ ]:
import re

X = 0
lines = []

for entry in data_points:
    if ('gpt3.5-score' in entry):
        numbers = re.findall(r'\d+', entry['gpt3.5-score'].strip())
        
        if len(numbers)==1:
            score = int(numbers[0])

        else:
            tmp_lines = entry['gpt3.5-score'].strip().split('\n')
            tmp_boolean = True

            for line in lines:
                if 'passage 3 score:' in line:
                    score = int(line.replace('passage 3 score:', '').strip())
                    tmp_boolean = False
                    break
                
                elif 'passage 3:' in line:
                    score = int(line.replace('passage 3:', '').strip())
                    tmp_boolean = False
                    break
            if tmp_boolean == False:
                score = int(entry['gpt3.5-score'].strip())
                X +=1

        line = entry['turn_id'] + '\t0\t'+ entry['doc_id']+ '\t'+ str(score) + '\n'
        lines.append(line)
        
print(X)

with open(output_text_path, 'w') as f:
    f.writelines(lines)  
    